In [ ]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/WeedDetectionDataset/"
CSV_PATH = "/content/drive/MyDrive/labels.csv"

df = pd.read_csv(CSV_PATH)

image_paths = []
labels = []

for _, row in df.iterrows():
    img_path = os.path.join(DATA_DIR, row['Filename'])
    if os.path.exists(img_path):
        image_paths.append(img_path)
        labels.append(row['Label'])
    else:
        print(f"Missing image: {img_path}")

train_paths, test_paths, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

label_to_species = dict(zip(df['Label'], df['Species'])) if 'Species' in df.columns else None




Mounted at /content/drive


In [ ]:
species_list = df.drop_duplicates('Label').sort_values('Label')['Species'].tolist()

In [ ]:
class DeepWeedsDatasetTF(tf.data.Dataset):
    def __new__(cls, image_paths, labels, img_size=(224, 224), batch_size=32, shuffle=True):
        num_classes = len(set(labels))

        def _load_and_preprocess(path, label):
            image = tf.io.read_file(path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, img_size)
            image = tf.cast(image, tf.float32) / 255.0
            label = tf.one_hot(label, num_classes)
            return image, label

        ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
        if shuffle:
            ds = ds.shuffle(buffer_size=1000)
        ds = ds.map(_load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return ds

In [ ]:
train_ds = DeepWeedsDatasetTF(train_paths, y_train)
test_ds = DeepWeedsDatasetTF(test_paths, y_test, shuffle=False)


In [ ]:
print("Train batches:", len(train_ds))
print("Test batches:", len(test_ds))

Train batches: 197
Test batches: 50


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

IMG_SIZE = (224, 224, 3)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(9, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

num_epochs = 20

loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

model.optimizer = optimizer

train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

train_acc_list = []
val_acc_list = []
epochs_list = []

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_acc_metric.update_state(labels, predictions)
    return loss

@tf.function
def val_step(images, labels):
    predictions = model(images, training=False)
    val_acc_metric.update_state(labels, predictions)

    for epoch in range(num_epochs):

        train_acc_metric.reset_state()
        val_acc_metric.reset_state()


        for images, labels in train_ds:
            train_step(images, labels)

        train_accuracy = train_acc_metric.result() * 100


        for images, labels in test_ds:
            val_step(images, labels)

        val_accuracy = val_acc_metric.result() * 100


        train_acc_list.append(train_accuracy.numpy())
        val_acc_list.append(val_accuracy.numpy())
        epochs_list.append(epoch + 1)

        print(f"Epoch [{epoch + 1}/{num_epochs}]")
        print(f"Train Accuracy: {train_accuracy:.2f}% | Val Accuracy: {val_accuracy:.2f}%")



In [ ]:

correct = 0
total = 0

for images, labels in test_ds:
    predictions = model(images, training=False)
    pred_labels = tf.argmax(predictions, axis=1)
    true_labels = tf.argmax(labels, axis=1)

    correct += tf.reduce_sum(tf.cast(pred_labels == true_labels, tf.float32)).numpy()
    total += labels.shape[0]

print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

all_preds = []
all_labels = []

for images, labels in test_ds:
    predictions = model(images, training=False)


    pred_labels = tf.argmax(predictions, axis=1)


    true_labels = tf.argmax(labels, axis=1)

    all_preds.extend(pred_labels.numpy())
    all_labels.extend(true_labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=list(label_to_species.values())))


In [ ]:
model_path = "/content/resnet50.keras"

resnet_model.save(model_path)

print(f"Model saved at {model_path}")

In [ ]:
import shutil
CSV_PATH = "/content/drive/MyDrive/labels.csv"
IMG_DIR = "/content/drive/MyDrive/WeedDetectionDataset/"
OUTPUT_DIR = "/content/deepweed-yolo/"


for split in ['train', 'val']:
    os.makedirs(f"{OUTPUT_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/labels/{split}", exist_ok=True)


df = pd.read_csv(CSV_PATH)


train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Label'], random_state=42)


def process_split(split_df, split_name):
    for _, row in split_df.iterrows():
        filename = row['Filename']
        label = int(row['Label'])


        src_img_path = os.path.join(IMG_DIR, filename)
        dst_img_path = os.path.join(OUTPUT_DIR, f"images/{split_name}/{filename}")
        dst_label_path = os.path.join(OUTPUT_DIR, f"labels/{split_name}/{filename.replace('.jpg', '.txt')}")


        if os.path.exists(src_img_path):
            shutil.copy(src_img_path, dst_img_path)


            with open(dst_label_path, 'w') as f:
                f.write(f"{label} 0.5 0.5 1.0 1.0\n")
        else:
            print(f"Missing image: {src_img_path}")


process_split(train_df, "train")
process_split(val_df, "val")


class_names = df.drop_duplicates("Label").sort_values("Label")["Species"].tolist()

yaml_path = os.path.join(OUTPUT_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"path: {OUTPUT_DIR}\n")
    f.write(f"train: images/train\n")
    f.write(f"val: images/val\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

print(" YOLO dataset ready!")
print(" Output directory:", OUTPUT_DIR)
print(" data.yaml created at:", yaml_path)

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Loading a YOLOv11 model
model = YOLO("yolo11n.pt")

# Training on deepweed
model.train(data='/content/deepweed-yolo/data.yaml', epochs=10,imgsz=640)

# Predict
results = model.predict(source='/content/deepweed-yolo/images/val', save=True)

In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import random

pred_dir = "/content/runs/detect/train"
all_images = [f for f in os.listdir(pred_dir) if f.endswith("_pred.jpg")]
random_imgs = random.sample(all_images, min(5, len(all_images)))

plt.figure(figsize=(15, 10))
for i, img_name in enumerate(random_imgs):
    img_path = os.path.join(pred_dir, img_name)
    img = Image.open(img_path)

    plt.subplot(1, len(random_imgs), i + 1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(img_name)
plt.tight_layout()
plt.show()

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')

metrics = model.val(data='/content/deepweed-yolo/data.yaml')

print(metrics)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

img = Image.open('content/runs/detect/train/results.png')
plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image as keras_image


yolo_weights_path = "/content/runs/detect/train/weights/best.pt"
resnet_model_path = "/content/resnet50.keras"
img_path = "/content/drive/MyDrive/WeedDetectionDataset/20160928-141437-0.jpg"
class_names = species_list


yolo_model = YOLO(yolo_weights_path)


results = yolo_model(img_path)
image = Image.open(img_path).convert("RGB")


resnet_model = tf.keras.models.load_model(resnet_model_path)


for i, box in enumerate(results[0].boxes.xyxy.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])


    cropped_img = image.crop((x1, y1, x2, y2)).resize((224, 224))


    img_array = keras_image.img_to_array(cropped_img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)


    preds = resnet_model.predict(img_array, verbose=0)
    pred_class = class_names[np.argmax(preds[0])]

    print(f"Box {i+1}: ({x1}, {y1}, {x2}, {y2}) → {pred_class}")


In [ ]:
from PIL import Image, ImageDraw, ImageFont
image_draw = image.copy()
draw = ImageDraw.Draw(image_draw)

try:
    font = ImageFont.truetype("arial.ttf", size=16)
except:
    font = ImageFont.load_default()

for i, box in enumerate(results[0].boxes.xyxy.cpu().numpy()):
    x1, y1, x2, y2 = map(int, box[:4])


    cropped_img = image.crop((x1, y1, x2, y2)).resize((224, 224))
    img_array = keras_image.img_to_array(cropped_img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)


    preds = resnet_model.predict(img_array, verbose=0)
    pred_label = np.argmax(preds[0])
    pred_name = class_names[pred_label]
    confidence = np.max(preds[0])

    label_text = f"{pred_label}: {pred_name} ({confidence:.2f})"


    draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    text_position = (x1 + 5, max(0, y1 - 20))
    draw.text(text_position, label_text, fill="red", font=font)

plt.figure(figsize=(10, 8))
plt.imshow(image_draw)
plt.axis("off")
plt.title("YOLO Detections + ResNet50 Class Predictions")
plt.show()